# SC1015 Mini-Project: Spotify Music Recommender System

### By FDDA Group 5
Syed Ali Redha Alsagoff, Huang Yongjian, Ma Jinlin


## Our problem statement

Our simple question of what determines a user's music preference actually has important implications in real life! With our project, we aim to find out **how companies can maximise the use of machine learning techniques to recommend quality songs efficiently**. To streamline the efficiency aspect, we will also look into **what features make a good recommender system**, as well as **what model to use in a genre classification model**.
<br>
<br>
A good genre prediction model is important due to the **lack of robust datasets with labelled genres**. Even in some research literatures on music recommender systems, the teams made use of genre classification models in their approaches.
<br>
<br>
We believe that this is an important issue to tackle because of **the vast amount of features used in music recommender systems**. Novel approaches to recommender systems make use of Deep learning that engineer even more features for their system[1]. Advanced recommendation systems can make use of many features[2], in the realm of hundreds to even thousands of features to make their recommendations. Hence, to improve the efficiency and resource management, and possibly the quality of recommendations as well, **good feature selection** is paramount to any music streaming service company, such as Spotify.
<br>
<br>
We will be using different feature selection techniques and will be evaluating our model trained using different feature selection techniques.
<br>
<br>
Due to time and hardware and other resource constraints, we will be doing our project using a smaller set of features than what is typically used. Larger models make use of more features such as lyric data (making use of tokenisers), audio data (making use of RNNs and LSTMs) and user-item correlation features, and more. However, the techniques that we use here would be **easily extendable** to datasets that make use of features on the scale of hundreds or even thousands.

## Installing relevant libraries

In [1]:
pip install --upgrade numpy scipy scikit-learn threadpoolctl

In [2]:
pip install spotipy

  Using cached spotipy-2.23.0-py3-none-any.whl (29 kB)
  Using cached redis-5.0.4-py3-none-any.whl (251 kB)


In [3]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

# Libraries for extracting dataset
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# Library and functions for getting pretrained models and mapper
import pickle

# Libraries for data preprocessing
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer

# Libraries for our autoencoder model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense


# Our dataset for recommender system

In this section, we will be detailing how we made our dataset to recommend songs to users. Spotify does not give access to all of its songs through its API, so we made our own dataset, using popular songs from a variety of genres for maximum coverage. To provide some simplicity, we constrained to system to recommend only English songs.

To make the Spotify API call, please use **the client ID and secret key provided in the NTULearn Submission**.  This is necessary for subsequent cells (

### Function to extract playlist information


In [36]:
def fetch_playlist_tracks(playlist_id, sp):
    track_uris = []
    artist_uris = []
    track_names = []
    artist_names = []
    track_popularities = []
    release_years = []
    explicit_statuses = []
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    for item in tracks:
        track = item['track']
        if track:
            track_uris.append(track['uri'])
            track_names.append(track['name'])
            artist_uris.extend([artist['uri'] for artist in track['artists']])
            artist_names.append(track['artists'][0]['name'] if track['artists'] else 'Unknown')
            track_popularities.append(track['popularity'])
            explicit_statuses.append(track['explicit'])
            release_date = track['album']['release_date']
            release_year = release_date.split("-")[0] if release_date else 'Unknown'
            release_years.append(release_year)
    return track_uris, track_names, artist_names, track_popularities, release_years, explicit_statuses, list(set(artist_uris))

def get_audio_info_from_playlist(playlist_id, sp):
    tracks, track_names, artist_names, popularities, release_years, explicit_statuses, artists = fetch_playlist_tracks(playlist_id, sp)
    combined_data = []

    for i in range(0, len(tracks), 50):
        track_interval = tracks[i:i+50]
        track_name_interval = track_names[i:i+50]
        artist_name_interval = artist_names[i:i+50]
        popularity_interval = popularities[i:i+50]
        release_year_interval = release_years[i:i+50]
        explicit_status_interval = explicit_statuses[i:i+50]

        if not track_interval:
            continue

        try:
            audio_features = sp.audio_features(track_interval)
        except spotipy.SpotifyException as e:
            if e.http_status == 429:
                print(f"Rate limit exceeded, sleeping for {retry_after} seconds")
                time.sleep(retry_after)
                # Exponential Backoff to comply with Spotify rate limits
                retry_after = min(retry_after * 2, 64)
            elif e.http_status == 400:
                print(f"Error with batch {i} to {i + len(track_interval) - 1}: {e.msg}, skipping...")
                continue
            else:
                print(f"Unhandled error: {e}")
                continue

        audio_features_batch = [
            [track_name_interval[j], artist_name_interval[j], popularity_interval[j], release_year_interval[j], explicit_status_interval[j]] +
            [v for k, v in d.items() if k not in ['type', 'id', 'uri', 'track_href', 'analysis_url']]
            for j, d in enumerate(audio_features) if d is not None
        ]

        for features in audio_features_batch:
            combined_data.append(features)

    columns = ['Track Name', 'artists', 'popularity', 'release_year', 'explicit', 'danceability', 'energy', 'key', 'loudness',
               'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
    return pd.DataFrame(combined_data, columns=columns)

# Please use the developer ID and secret attached in the NTULearn submission
client_id = ''
client_secret = ''

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10, retries=5, status_forcelist=(429, 500, 502, 503, 504), backoff_factor=0.3)

### Extracting the dataset

In [11]:
full_dataset_df1 = pd.read_csv('spotify_playlist_dataset1.csv')
full_dataset_df2 = pd.read_csv('spotify_playlist_dataset2.csv')
full_dataset_df = pd.concat([full_dataset_df1,full_dataset_df2], ignore_index=True)
full_dataset_df.drop_duplicates(subset=['Track Name'], keep='first',inplace=True)
print(full_dataset_df.shape)

(11794, 19)


In [ ]:
# Feel free to running the code but sometimes there might be error due too too many request, which happens when Spotify API temporarily ban the account from making API calls
# test_df = get_audio_info_from_playlist("(insert the playlist uri here)",sp)

If you'd like, you can uncomment the code above and try adding in different playlist uris to extract their information into the test_df DataFrame. You can get playlist uris by simply going to your playlist on desktop and hovering share. If you press the option (for Mac) or control (for Windows) button, you can see the copy link button change to copy playlist URI.

## Data Cleaning

In [12]:
# Check the data in the playlist
full_dataset_df.info()
full_dataset_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 11794 entries, 0 to 17579
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        11794 non-null  int64  
 1   Track Name        11793 non-null  object 
 2   Artist Name       11793 non-null  object 
 3   Popularity        11794 non-null  int64  
 4   Release Year      11794 non-null  int64  
 5   Explicit          11794 non-null  bool   
 6   Danceability      11794 non-null  float64
 7   Energy            11794 non-null  float64
 8   Key               11794 non-null  int64  
 9   Loudness          11794 non-null  float64
 10  Mode              11794 non-null  int64  
 11  Speechiness       11794 non-null  float64
 12  Acousticness      11794 non-null  float64
 13  Instrumentalness  11794 non-null  float64
 14  Liveness          11794 non-null  float64
 15  Valence           11794 non-null  float64
 16  Tempo             11794 non-null  float64
 17

Unnamed: 0          0
Track Name          1
Artist Name         1
Popularity          0
Release Year        0
Explicit            0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
Duration_ms         0
Time Signature      0
dtype: int64

As seen above, we still have some NaN values and an irrelevant column, we will now remove them.

In [13]:
# Remove irrelevant columns and drop any rows with NaN values
full_dataset_df.drop("Unnamed: 0", axis=1, inplace=True)
full_dataset_df.dropna(inplace=True)
full_dataset_df.info()
full_dataset_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 11793 entries, 0 to 17579
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        11793 non-null  object 
 1   Artist Name       11793 non-null  object 
 2   Popularity        11793 non-null  int64  
 3   Release Year      11793 non-null  int64  
 4   Explicit          11793 non-null  bool   
 5   Danceability      11793 non-null  float64
 6   Energy            11793 non-null  float64
 7   Key               11793 non-null  int64  
 8   Loudness          11793 non-null  float64
 9   Mode              11793 non-null  int64  
 10  Speechiness       11793 non-null  float64
 11  Acousticness      11793 non-null  float64
 12  Instrumentalness  11793 non-null  float64
 13  Liveness          11793 non-null  float64
 14  Valence           11793 non-null  float64
 15  Tempo             11793 non-null  float64
 16  Duration_ms       11793 non-null  int64  
 17

Track Name          0
Artist Name         0
Popularity          0
Release Year        0
Explicit            0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
Duration_ms         0
Time Signature      0
dtype: int64

In [14]:
full_dataset_df = full_dataset_df.rename(columns={"Acousticness": "acousticness", "Danceability": "danceability","Energy":"energy","Instrumentalness":"instrumentalness","Liveness":"liveness","Loudness":"loudness","Popularity":"popularity","Speechiness":"speechiness","Tempo":"tempo","Valence":"valence","Duration_ms":"duration_ms","Time Signature":"time_signature","Key":"key","Mode":"mode", "Artist Name": "artists","Release Year":"release_year","Explicit":"explicit"})
full_dataset_df.columns

Index(['Track Name', 'artists', 'popularity', 'release_year', 'explicit',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature'],
      dtype='object')

## Data Preparation - Feature Engineering
Firstly, we will be doing feature engineering, which will be the genre feature. As explained in the first notebook (Part 1: Genre Classification Model), We believe that this will be an essential feature in recommending tracks. We will be making use of the model and mapper from the Genre Classification Notebook to get this feature.

In [18]:
class CustomMultiLabelBinarizer:
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    # Fit the Scikit-Learn MultiLabelBinarizer to the data
    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self


    # Transform the data while ignoring unseen labels
    def transform(self, X):
        try:
            return self.mlb.transform(X)
        except ValueError as e:  # Handle the case where there are unseen labels in X (test dataset) compared to fitted labels
            unseen = set(x for sublist in X for x in sublist) - set(self.mlb.classes_)
            X_filtered = [[x for x in sublist if x not in unseen] for sublist in X]
            return self.mlb.transform(X_filtered)



    # Fit and transfrom data
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)

    # Return feature names for output columns
    def get_feature_names(self):
        return self.mlb.classes_

In [19]:
# Import pretrained models and mapper from other notebook
with open('multinomial_logistic_regression_model.pkl', 'rb') as file:
    multinomial_logistic_regression_model = pickle.load(file)
with open('mapper.pkl', 'rb') as file:
    mapper = pickle.load(file)

# Function to extract the values from the DataFrames
def get_genres(df):
    df = df.copy()
    df['artists'] = df['artists'].str.split(';')
    df.drop(['Track Name', 'release_year','tempo','duration_ms'], axis=1, inplace=True)
    X = mapper.transform(df)
    X.columns = [col.replace('artists_', '').replace('[', '').replace(']', '').replace('<', '') for col in X.columns]
    X = X.loc[:, ~X.columns.duplicated()]
    genres = multinomial_logistic_regression_model.predict(X)
    return genres

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.2 when using version 1.4.2. This might lead to breaking code

In [20]:
# Add genres to the dataset
full_dataset_df["genre"] = get_genres(full_dataset_df)
last_rows_index = full_dataset_df.tail(1703).index
full_dataset_df.loc[last_rows_index, 'genre'] = get_genres(full_dataset_df.loc[last_rows_index])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['$uicideboy$', '*NSYNC', '+44', '112', '1999 WRITE THE FUTURE', '2 Chainz', '22 the Light', '2nd II None', '3k9x', '3slow2', '4 Non Blondes', '4batz', '6arelyhuman', '6ix9ine', '702', '730RARRI', '80purppp', '86Casino', '88rising', '8percent', '9infidell', 'A Boogie Wit da Hoodie', 'A Boy and His Kite', 'A Perfect Circle', 'A Tribe Called Quest', 'A!MS', 'A-Wall', 'ABBA', 'ADOY', 'AFROJACK', 'AGA', 'ALEXSUCKS', 'ALTÉGO', 'ALYA', 'ANTH', 'ASTN', 'ATARASHII GAKKO!', 'AVAION', 'Aaliyah', 'Aaron May', 'Aaron Smith', 'Aaron Tippin', 'Abby Anderson', 'Abe Parker', 'Abigail Bleu', 'Abigail Fierce', 'Abrina', 'Ace Hood', 'Ace of Base', 'Acoustic Vibe', 'Adam & The Ants', 'Adam Cashen', 'Adam Hicks', 'Adam Lambert', 'Adam Wheeler', 'Addy Lamaute', 'Adele', 'Adina Howard', 'Adrian Marcel', 'Adriano Spampanato', 'Afgan', 'Afroman', 'Agnes', 'Ahmad Jamal Trio', 'Ahzee', 'Aimee Carty', 'Air 

In [21]:
# Check for any rows where genres are null
full_dataset_df["genre"].isnull().sum()

0

As you can see we successfully got the genres for all rows. After this, we can save our dataframe into csv file. Do note that we already attached 'spotify_playlist_dataset.csv' in the repository, so you do not need to run this.

In [48]:
#full_dataset_df.to_csv('spotify_playlist_dataset.csv')

## Data Preparation - Data Preprocessing

We will now be preprocessing our data, by making all categorical features numeric. We will be making use of one-hot encoding to do so. <br>

Then, we will make use of an autoencoder model to further reduce the dimensionality and improve the richness of the features of our model. <br>

Then, we will be scaling and normalising our features before finally making use of them in our model.

In [22]:
scaler = StandardScaler()
categorical_features = ['key', 'mode', 'genre','explicit']
numerical_features = ['duration_ms', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','popularity']
full_dataset_df = pd.get_dummies(full_dataset_df, columns=categorical_features)

# Listout one-Hot encoded features
final_categorical_features = ['key_0','key_1','key_2','key_3','key_4','key_5','key_6','key_7','key_8','key_9','key_10','key_11','mode_0','mode_1', 'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6','explicit_False','explicit_True']
for column in final_categorical_features:
    full_dataset_df[column] = full_dataset_df[column].astype(int)

full_dataset_df.info()

scaled_full_dataset_df = full_dataset_df.copy()
scaled_full_dataset_df[numerical_features] = scaler.fit_transform(scaled_full_dataset_df[numerical_features])

<class 'pandas.core.frame.DataFrame'>
Index: 11793 entries, 0 to 17579
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        11793 non-null  object 
 1   artists           11793 non-null  object 
 2   popularity        11793 non-null  int64  
 3   release_year      11793 non-null  int64  
 4   danceability      11793 non-null  float64
 5   energy            11793 non-null  float64
 6   loudness          11793 non-null  float64
 7   speechiness       11793 non-null  float64
 8   acousticness      11793 non-null  float64
 9   instrumentalness  11793 non-null  float64
 10  liveness          11793 non-null  float64
 11  valence           11793 non-null  float64
 12  tempo             11793 non-null  float64
 13  duration_ms       11793 non-null  int64  
 14  time_signature    11793 non-null  int64  
 15  key_0             11793 non-null  int64  
 16  key_1             11793 non-null  int64  
 17

## Autoencoder Model

To achieve dimensionality reduction, one of the methods that we will be using is an **Autoencoder neural network**. An autoencoder is made out of 2 parts, an encoder and a decoder. The encoder section applies transformations onto the input data, **reducing the dimensionality of data** in each layer. This is done until it reaches a low-dimensional bottleneck. This bottleneck captures the essential features of the data. The decoder then tries to **reconstruct the original data** from this compressed representation. How different the resulting output is from the original is taken as the **loss function**.

### Unweighted autoencoder model
The Unweighted Autoencoder **does not assign different importances to different features** when calculating the loss during training. It treats all errors between the reconstructed output and the original input equally across the dataset. This is useful for dataset with features that have around the same importances and is generally simpler than the weighted variant.

In [23]:
# Exclude non-numeric features like 'Track Name', 'Artist Name' for input
input_dim = full_dataset_df.shape[1] - 2

# Encoder/Decoder definition (Hyperparameters were tuned such that optimal results were given)
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Compile the autoencoder
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Prepare data for training

# Use numeric data only
X_train = full_dataset_df.drop(['Track Name', 'artists'], axis=1)

# Convert to float32 to ensure compatibility with TensorFlow
X_train = X_train.astype('float32')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_split=0.2)

# Create a separate encoder model from the full autoencoder
unweighted_encoder = Model(input_layer, encoded)

Epoch 1/50
37/37 [==============================] - 2s 13ms/step - loss: 1576384000.0000 - val_loss: 1269431296.0000
Epoch 2/50
37/37 [==============================] - 0s 8ms/step - loss: 1576384000.0000 - val_loss: 1269431296.0000
Epoch 3/50
37/37 [==============================] - 0s 6ms/step - loss: 1576384000.0000 - val_loss: 1269431296.0000
Epoch 4/50
37/37 [==============================] - 0s 6ms/step - loss: 1576383872.0000 - val_loss: 1269431296.0000
Epoch 5/50
37/37 [==============================] - 0s 7ms/step - loss: 1576384000.0000 - val_loss: 1269431296.0000
Epoch 6/50
37/37 [==============================] - 0s 8ms/step - loss: 1576383872.0000 - val_loss: 1269431296.0000
Epoch 7/50
37/37 [==============================] - 0s 6ms/step - loss: 1576384000.0000 - val_loss: 1269431296.0000
Epoch 8/50
37/37 [==============================] - 0s 7ms/step - loss: 1576383872.0000 - val_loss: 1269431296.0000
Epoch 9/50
37/37 [==============================] - 0s 7ms/step - loss:

### Weighted autoencoder model

The Weighted Autoencoder can **assign different weights to features**, allowing the model to prioritise minimising the error cost of specific features over the others during training. In this autoencoder model, we will be adjusting the weights based on the feature importance in deciding a song's popularity that we found earlier in the second notebook (Part 2: Music Popularity Model).

The weights of **music genre, explicit lyrics, release year, instrumentalness** will be adjusted upwards while the weights of **key, mode and danceability** are adjusted downwards. Artist names are not included in our recommender system as it requires multilabel binarizer and will create too many features for our small-scaled recommender system.

We will be simulating this in the model by simply multiplying the values by a number greater than one for the more important features, and a number less than one for a less important feature.

In [24]:
# Exclude non-numeric features: 'Track Name', 'Artist Name' for input
input_dim = full_dataset_df.shape[1] - 2

# Encoder/Decoder definition (Hyperparameters were tuned such that optimal results were given)
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Compile the autoencoder
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Prepare data for training
# Use numeric data only
X_train = full_dataset_df.drop(['Track Name', 'artists'], axis=1)

# Convert to float32 to ensure compatibility with TensorFlow
X_train = X_train.astype('float32')

# Adjust the weights of different values for the model
# music genre, explicit lyrics, release year, instrumentalness are adjusted upwards
# key, mode and danceability are adjusted downwards
X_train[['key_0','key_1','key_2','key_3','key_4','key_5','key_6','key_7','key_8','key_9','key_10','key_11']] *= 0.98
X_train[['mode_0','mode_1']] *= 0.98
X_train['danceability'] *= 0.98
X_train['instrumentalness'] *= 1.02
X_train['release_year'] *= 1.02
X_train[['explicit_False','explicit_True']] *= 1.02
X_train['popularity'] *= 1.1
X_train[['genre_0','genre_1','genre_2','genre_3','genre_4','genre_5','genre_6']] *= 1.02

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_split=0.2)

# Create a separate encoder model from the full autoencoder
weighted_encoder = Model(input_layer, encoded)

Epoch 1/50
37/37 [==============================] - 2s 11ms/step - loss: 1576401024.0000 - val_loss: 1269447424.0000
Epoch 2/50
37/37 [==============================] - 0s 6ms/step - loss: 1576395008.0000 - val_loss: 1269435904.0000
Epoch 3/50
37/37 [==============================] - 0s 8ms/step - loss: 1576388352.0000 - val_loss: 1269435904.0000
Epoch 4/50
37/37 [==============================] - 0s 8ms/step - loss: 1576388352.0000 - val_loss: 1269435904.0000
Epoch 5/50
37/37 [==============================] - 0s 7ms/step - loss: 1576388352.0000 - val_loss: 1269435904.0000
Epoch 6/50
37/37 [==============================] - 0s 6ms/step - loss: 1576388352.0000 - val_loss: 1269435904.0000
Epoch 7/50
37/37 [==============================] - 0s 7ms/step - loss: 1576388480.0000 - val_loss: 1269435904.0000
Epoch 8/50
37/37 [==============================] - 0s 6ms/step - loss: 1576388480.0000 - val_loss: 1269435904.0000
Epoch 9/50
37/37 [==============================] - 0s 6ms/step - loss:

Now, we have done all the relevant preprocessing and preparation with our main dataset. We will now preprocess and prepare our user playlist dataset.

## Preprocessing and preparing user playlist data

In [55]:
# Extract the features of the user playlist
playlist_features_df = get_audio_info_from_playlist('spotify:playlist:4bT12S3KMsTp8ZSaqd2RYq', sp)
playlist_features_df['genre'] = get_genres(playlist_features_df)
playlist_features_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['Alex G', 'Beth', 'EMELINE', 'Kurt Hugo Schneider', 'Lewis Capaldi', 'Madilyn Bailey', 'NDPeeps', 'Zac Efron'] will be ignored
  warnings.warn(


,Track Name,artists,popularity,release_year,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,Someone You Loved,Lewis Capaldi,90,2019,False,0.501,0.405,1,-5.679,1,0.0319,0.7510,0.000000,0.105,0.4460,109.891,182161,4,5
1,Arcade,Duncan Laurence,76,2020,False,0.453,0.350,9,-11.639,0,0.0425,0.8270,0.001220,0.144,0.2790,72.017,185147,3,5
2,Talking to the Moon,Bruno Mars,84,2010,False,0.498,0.590,1,-4.721,0,0.0320,0.5110,0.000000,0.107,0.0784,145.867,217867,4,0
3,Titanium (feat. Sia),David Guetta,58,2012,False,0.604,0.787,0,-3.674,0,0.1030,0.0679,0.150000,0.127,0.3010,126.062,245040,4,0
4,"My Heart Will Go On - Love Theme from ""Titanic""",Céline Dion,75,1997,False,0.428,0.276,4,-11.729,1,0.0312,0.7320,0.000005,0.117,0.0382,99.195,280000,1,3


In [57]:
playlist_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        27 non-null     object 
 1   artists           27 non-null     object 
 2   popularity        27 non-null     int64  
 3   release_year      27 non-null     object 
 4   explicit          27 non-null     bool   
 5   danceability      27 non-null     float64
 6   energy            27 non-null     float64
 7   key               27 non-null     int64  
 8   loudness          27 non-null     float64
 9   mode              27 non-null     int64  
 10  speechiness       27 non-null     float64
 11  acousticness      27 non-null     float64
 12  instrumentalness  27 non-null     float64
 13  liveness          27 non-null     float64
 14  valence           27 non-null     float64
 15  tempo             27 non-null     float64
 16  duration_ms       27 non-null     int64  
 17 

In [58]:
# One-hot encoding the categorical features
playlist_features_df[final_categorical_features] = 0
categorical_features = ['key', 'mode', 'genre','explicit']
playlist_features_df = pd.get_dummies(playlist_features_df, columns=categorical_features)
# Not every value in the categorical features will be present, so to circumvent this
# We will be manually putting in every necessary column, and merge them
for col in final_categorical_features:
    cols_to_merge = [c for c in playlist_features_df.columns if c.startswith(col)]
    if len(cols_to_merge) > 1:
        # Make use of max function to effectively perform an OR operation across duplicates
        playlist_features_df[col] = playlist_features_df[cols_to_merge].max(axis=1)
playlist_features_df = playlist_features_df.loc[:,~playlist_features_df.columns.duplicated()].copy()

# ensure all correct values are numeric and not of type object
for column in (final_categorical_features + ["release_year"]):
    playlist_features_df[column] = playlist_features_df[column].astype(int)
playlist_features_df.info()

# Scale the playlist features for the non-autoencoder model
scaled_playlist_features_df = playlist_features_df.copy()
scaled_playlist_features_df[numerical_features] = scaler.fit_transform(scaled_playlist_features_df[numerical_features])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        27 non-null     object 
 1   artists           27 non-null     object 
 2   popularity        27 non-null     int64  
 3   release_year      27 non-null     int64  
 4   danceability      27 non-null     float64
 5   energy            27 non-null     float64
 6   loudness          27 non-null     float64
 7   speechiness       27 non-null     float64
 8   acousticness      27 non-null     float64
 9   instrumentalness  27 non-null     float64
 10  liveness          27 non-null     float64
 11  valence           27 non-null     float64
 12  tempo             27 non-null     float64
 13  duration_ms       27 non-null     int64  
 14  time_signature    27 non-null     int64  
 15  key_0             27 non-null     int64  
 16  key_1             27 non-null     int64  
 17 

In [59]:
playlist_features_df.head()

,Track Name,artists,popularity,release_year,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,mode_1,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,explicit_False,explicit_True
0,Someone You Loved,Lewis Capaldi,90,2019,0.501,0.405,-5.679,0.0319,0.7510,0.000000,...,1,0,0,0,0,0,1,0,1,0
1,Arcade,Duncan Laurence,76,2020,0.453,0.350,-11.639,0.0425,0.8270,0.001220,...,0,0,0,0,0,0,1,0,1,0
2,Talking to the Moon,Bruno Mars,84,2010,0.498,0.590,-4.721,0.0320,0.5110,0.000000,...,0,1,0,0,0,0,0,0,1,0
3,Titanium (feat. Sia),David Guetta,58,2012,0.604,0.787,-3.674,0.1030,0.0679,0.150000,...,0,1,0,0,0,0,0,0,1,0
4,"My Heart Will Go On - Love Theme from ""Titanic""",Céline Dion,75,1997,0.428,0.276,-11.729,0.0312,0.7320,0.000005,...,1,0,0,0,1,0,0,0,1,0


Now all that we need to do is to apply the Autoencoder and scale them.

In [60]:
unweighted_full_encoded = unweighted_encoder.predict(X_train)
weighted_full_encoded = weighted_encoder.predict(X_train)

unweighted_playlist_encoded = unweighted_encoder.predict(playlist_features_df.drop(['Track Name', 'artists'], axis=1).astype('float32'))
weighted_playlist_encoded = weighted_encoder.predict(playlist_features_df.drop(['Track Name', 'artists'], axis=1).astype('float32'))

weighted_full_encoded_scaled = scaler.fit_transform(weighted_full_encoded)
unweighted_full_encoded_scaled = scaler.fit_transform(unweighted_full_encoded)

unweighted_playlist_encoded_scaled = scaler.transform(unweighted_playlist_encoded)
weighted_playlist_encoded_scaled = scaler.transform(weighted_playlist_encoded)

1/1 [==============================] - 0s 21ms/step


# Our Models

In this section, we will be making different models using the different features and techniques we found earlier. Our baseline model will be a simple **K Nearest Neighbours algorithm** to recommend songs to users. While the novel approach is a combination of collaborative, content-based and deep learning approaches, the lack of available good datasets and hardware constraints made us stick to a **simpler, content-based filtering algorithm** as our recommender system.
<br>
<br>
Our 5 models are as follows:
1. Baseline model with using all features and default feature weights
2. Baseline model with using all features and adjusted feature weights (based on insights from Part 2: Music Popularity Model)
3. Baseline model with the worst feature weights dropped (based on insights from Part 2: Music Popularity Model)
4. Baseline model with the Unweighted Autoencoder
5. Baseline model with the Weighted Autoencoder

We will not only be testing how well the models do with the dimensionality reduction, but also how well the model does if we add weights to the features, and seeing if it helps to improve the quality of our predictions.

### 1. Our baseline model

In [61]:
from sklearn.neighbors import NearestNeighbors

# Initialize and Train the NearestNeighbors Model
similarity_metric = 'cosine'

# Fit and Train model
nn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric=similarity_metric)
nn_model.fit(scaled_full_dataset_df.drop(['Track Name','artists'], axis=1).values)

# Calculate the average features for the playlist
average_features = scaled_playlist_features_df.drop(['Track Name','artists'], axis=1).mean(axis=0).to_numpy().reshape(1, -1)

# Query the nearest neighbors model using the average feature vector
distances, indices = nn_model.kneighbors(average_features)
recommended_tracks = full_dataset_df.iloc[indices[0]]
for i in range(5):
    print(f"{recommended_tracks['Track Name'].iloc[i]} by {recommended_tracks['artists'].iloc[i]}")

Locked Away (feat. Adam Levine) by R. City
Ironic - 2015 Remaster by Alanis Morissette
Love Somebody by Maroon 5
The Best Day (Taylor’s Version) by Taylor Swift
Welcome To My Island by Caroline Polachek


### 2. Baseline model with adjusted feature weights

Here we will be making use of our baseline model, but adding feature importances to our features through multiplying a feature that is more important with a value greater than one and a feature that is less important with a value less than one.

In [62]:
from sklearn.neighbors import NearestNeighbors

# Set the similarity metric
similarity_metric = 'cosine'

# Set weights
# music genre, explicit lyrics, release year, instrumentalness are adjusted upwards
# key, mode and danceability are adjusted downwards
weights = {'genre_0': 1.2, 'genre_1': 1.2, 'genre_2': 1.2, 'genre_3': 1.2, 'genre_4': 1.2, 'genre_5': 1.2, 'genre_6': 1.2, 'explicit_False': 1.1, 'explicit_True': 1.1, 'release_year': 1.1, 'instrumentalness': 1.1, \
           'key_0': 0.8, 'key_1': 0.8, 'key_2': 0.8, 'key_3': 0.8, 'key_4': 0.8, 'key_5': 0.8, 'key_6': 0.8 , 'key_7': 0.8, 'key_8': 0.8, 'key_9': 0.8, 'key_10': 0.8, 'key_11': 0.8, \
           'mode_0': 0.9, 'mode_1': 0.9, 'danceability': 0.9}

# Apply weights
for feature, weight in weights.items():
    scaled_full_dataset_df[feature] *= weight


# Initialize and Train the NearestNeighbors Model with the chosen metric
nn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric=similarity_metric)
nn_model.fit(scaled_full_dataset_df.drop(['Track Name','artists'], axis=1).values)

average_features = scaled_playlist_features_df.drop(['Track Name', 'artists'],axis=1).values.mean(axis=0).reshape(1, -1)
# Query the nearest neighbors model using the average feature vector
distances, indices = nn_model.kneighbors(average_features)
recommended_tracks = full_dataset_df.iloc[indices[0]]
for i in range(5):
    print(f"{recommended_tracks['Track Name'].iloc[i]} by {recommended_tracks['artists'].iloc[i]}")

Locked Away (feat. Adam Levine) by R. City
Ironic - 2015 Remaster by Alanis Morissette
Ironic by Alanis Morissette
Fight For Your Right by Beastie Boys
How Do You Sleep? by Sam Smith


### 3. Baseline models with features dropped

Here, we will only be using the top 5 features (excluding artists) that were found in the Music Popularity Model, namely genre, explicit, instrumentalness, loudness and release year.

In [63]:
from sklearn.neighbors import NearestNeighbors

# Set the similarity metric
similarity_metric = 'cosine'

# Initialize and Train the NearestNeighbors Model with the chosen metric
nn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric=similarity_metric)
nn_model.fit(scaled_full_dataset_df[['genre_0', 'genre_1', 'genre_2', 'genre_3','genre_4','genre_5','genre_6','explicit_True','explicit_False','instrumentalness','loudness','release_year']].values)
average_features = scaled_playlist_features_df[['genre_0', 'genre_1', 'genre_2', 'genre_3','genre_4','genre_5','genre_6','explicit_True','explicit_False','instrumentalness','loudness','release_year']].values.mean(axis=0).reshape(1, -1)
# Query the nearest neighbors model using the average feature vector
distances, indices = nn_model.kneighbors(average_features)
recommended_tracks = full_dataset_df.iloc[indices[0]]
for i in range(5):
    print(f"{recommended_tracks['Track Name'].iloc[i]} by {recommended_tracks['artists'].iloc[i]}")

Tiny Moves by Bleachers
Call Me Up by daydreamers
Afterthought by Joji
Why Don't You by Cleo Sol
The Night Begins to Shine by B.E.R.


### 4. Baseline model with Unweighted Autoencoder

Our model, with the unweighted autoencoder used to reduce dimensionality.

In [64]:
from sklearn.neighbors import NearestNeighbors

# Set the similarity metric
similarity_metric = 'cosine'

# Initialize and Train the NearestNeighbors Model with the chosen metric
nn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric=similarity_metric)
nn_model.fit(unweighted_full_encoded_scaled)


average_features = unweighted_playlist_encoded_scaled.mean(axis=0).reshape(1, -1)
# Query the nearest neighbors model using the average feature vector
distances, indices = nn_model.kneighbors(average_features)
recommended_tracks = full_dataset_df.iloc[indices[0]]
for i in range(5):
    print(f"{recommended_tracks['Track Name'].iloc[i]} by {recommended_tracks['artists'].iloc[i]}")

Place Your Hands by Reef
I Fall In Love Too Easily by Chet Baker
I Think I'm Paranoid by Garbage
Long Train Runnin' by The Doobie Brothers
The Way You Look Tonight by Frank Sinatra


### 5. Baseline model with Weighted Autoencoder

Our model, with the weighted autoencoder used to reduce dimensionality and to try and improve the quality of recommendations.

In [65]:
from sklearn.neighbors import NearestNeighbors

# Set the similarity metric, we chose cosine as seemed to give the best results
similarity_metric = 'cosine'

# Initialize and Train the NearestNeighbors Model with the chosen metric
nn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric=similarity_metric)
nn_model.fit(weighted_full_encoded_scaled)

# Calculate the average features for the playlist
average_features = weighted_playlist_encoded_scaled.mean(axis=0).reshape(1, -1)

# Query the nearest neighbors model using the average feature vector
distances, indices = nn_model.kneighbors(average_features)
recommended_tracks = full_dataset_df.iloc[indices[0]]
for i in range(5):
    print(f"{recommended_tracks['Track Name'].iloc[i]} by {recommended_tracks['artists'].iloc[i]}")

Go Your Own Way - 2004 Remaster by Fleetwood Mac
I've Got You Under My Skin - Remastered 1998 by Frank Sinatra
I'm with You by Avril Lavigne
Tha Crossroads by Bone Thugs-N-Harmony
Brighter by Paramore


# Results

After making our models, we will be evaluating them based on user feedback. We got 10 users to try the recommender system, and got them to rate the quality of the top 5 recommendations. Here are our results:
<br>
(We can say here for our different model averages (if you have time maybe try the models out and see what numbers are more representative):
<br>
1. Baseline model with adjusted weights: **7.04**
2. Baseline model with the weighted autoencoder: 6.9
3. Baseline model: 6.74
4. Baseline model with the unweighted Autoencoder: 6.06
5. Baseline models with worst features dropped: 4.7

![title](img/survey.png)

# Data-driven Insights and Recommendations

As you can see, while there are still areas for improvement for the quality of predictions across the board, the resultant recommendations were overall satisfactory. From our data, we can make a number of insights and recommendations to small-scale music streaming companies.

1. **The baseline model with adjusted weights** based on features that we found to be important for song popularity **performed the best** amongst the five models. However, we also note that the use of **Weighted Autoencoders** can help **improve efficiency without significant performance cost** when we are making use of numerical metadata. It seems that for a small-scaled music recommender system, dimensionality reduction techniques like Weighted Autoencoder can be paired with feature enhancement/ model enhancement strategies to reach compromise between cost and performance.

2. **Make use of a more robust recommender system**. While our results are not too bad, a simple content-based filtering algorithm is not as effective in dealing with **subjective preferences** compared to a model that uses **both content-based and collaborative filtering**. Implementation of collaborative filtering, which seeks to filter out music suggestions to a user based on how similar users react, can help to enhance our model further to personalise recommendations.

3. **Broaden the types of data used**. We noticed that our system only uses **structured data** for training the recommender system. Novel approaches will make use of **lyric and audio data** to enhance feature selection and potentially improve the quality of recommendations to users. However, this will come at the expense of **more compute**.

4. **To balance cost and performance**, for systems which take in unstructured data such as unstructured text-based lyrics data and audio data, we recommend similar dimensionality reduction techniques such as the use of **Mel-Frequency Cepstral Coefficients (MFCCs) for audio data** [3], which is a method of encoding audio data and the use of **Singular Value Decomposition (SVD) for textual data (like lyrics)**.[4]
<br>
We believe that with these recommendations, a smaller company with less resources can make make a and a good and computationally efficient model to recommend songs to users. Some of our insights could also possibly help larger companies as well, to make their recommendations more robust.


### From Part 1: Genre Classification Model
5. **Simpler models sometimes achieve better results**. The Multinomial Logistic Model beat the likes of sophisticated models such as XGBoost and Random Forest to give us the best classification metrics (e.g. accuracy, AOC score etc)

### From Part 2: Music Popularity Model
6. **Popularity of a song** is more significantly influenced by the artists due to their accumulated fame and genre is shaped by one's preference in music. It is more interesting to note that **newer songs tend to be more popular** since they leave a stronger impression to current listeners. This is also boosted by the **digitalisation of music industry** as newer songs could be distributed more easily online. Presence of explicit lyrics could attract **eyeball attention** and influence the song popularity. Instrumentalness which measures the presence of vocal (higher implies lesser vocal content) is also important in deciding song popularity as most popular songs **generally contain vocals**.

# Citations

[1] M. Schedl, ‘Deep Learning in Music Recommendation Systems’, Front. Appl. Math. Stat., vol. 5, doi: 10.3389/fams.2019.00044.
<br>
[2] F. Ricci, L. Rokach, and B. Shapira, Eds., Recommender Systems Handbook. New York, NY: Springer US, 2022. doi: 10.1007/978-1-0716-2197-4.
<br>
[3] V. Tiwari, "MFCC and its applications in speaker recognition," International Journal on Emerging Technologies, vol. 1, no. 1, pp. 19-22, 2010.
<br>
[4] P. C. Hansen, "The truncatedSVD as a method for regularization," BIT Numerical Mathematics, vol. 27, no. 4, pp. 534-553, 1987. doi: 10.1007/BF01937276.